# Tarea: Bases de Datos Vectoriales

## Stiven Saldaña

# Ejercicio 7: Bases de Datos Vectoriales
**Objetivo de la práctica**

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

**Parte 0: Carga del Corpus**

Vamos a utilizar la API de Kaggle para acceder al dataset Wikipedia Text Corpus for NLP and LLM Projects

El corpus está disponible desde este link

# Cargar el corpus


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [2]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

Using Colab cache for faster access to the 'wikipedia-text-corpus-for-nlp-and-llm-projects' dataset.


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


# Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este link

# Actividad

1. Normalizar el corpus
2. Definir una función chunk_text, y dividir los textos en chunks.
3. Generar embeddings por cada chunk

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [4]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

# Embeddings (N x D)
# normalize_embeddings=True es útil si usarás cosine similarity
embeddings = model.encode(
    passages,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

embeddings.shape, embeddings.dtype

In [7]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [8]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

# Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este link

# Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una query

In [9]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 103.4 MB/s eta 0:00:00


In [10]:
import faiss

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
k = 5
D, I = index.search(query_vec, k)

print(f"Top {k} indices:", I)
print(f"Distancias:", D)

Top 5 indices: [[10176     1 10177 37406 71872]]
Distancias: [[0.25930297 0.27639934 0.3197968  0.32173356 0.32282233]]


# Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

**Objetivo**

Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de metadata y filtros.

# Qué debes implementar

1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
* dimensión D (la de tus embeddings)
* métrica (cosine o L2)
3. Insertar:
* id
* embedding
* payload (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
* query_embedding
* k

# Inputs esperados (ya definidos arriba en el notebook)

* embeddings: matriz N x D (float32)
* texts: lista de N strings
* metadatas: lista de N dicts (opcional)
* query_text: string
* query_embedding: vector 1 x D

# Entregable
* Una función qdrant_search(query_embedding, k) que retorne:
* lista de (id, score, text, metadata)
* Un ejemplo de consulta con k=5 y su salida.


In [11]:
!pip install qdrant-client -q

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# inicializar cliente en memoria
client = QdrantClient(":memory:")
COLLECTION_NAME = "wikipedia_chunks"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 10.5 MB/s eta 0:00:00


In [12]:
# crear la colección
dim = embeddings.shape[1]
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=dim, distance=Distance.COSINE),
)

points = []
for idx, row in chunks_df.iterrows():
    points.append(
        PointStruct(
            id=idx,
            vector=embeddings[idx].tolist(),
            payload={
                "text": row["text"],
                "doc_id": row["doc_id"],
                "chunk_id": row["chunk_id"]
            }
        )
    )

# cargar los datos
client.upsert(
    collection_name=COLLECTION_NAME,
    points=points
)

print(f"Colección '{COLLECTION_NAME}' lista con {len(points)} puntos.")

/tmp/ipython-input-1850645991.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Colección 'wikipedia_chunks' lista con 79104 puntos.


/tmp/ipython-input-1850645991.py:23: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 79104 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client.upsert(


In [13]:
!pip install --upgrade qdrant-client -q

In [16]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

#inicializar
client = QdrantClient(":memory:")
COLLECTION_NAME = "wikipedia_chunks"

#crear colección
dim = embeddings.shape[1]
if not client.collection_exists(COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=dim, distance=Distance.COSINE),
    )

#insertar datos
points = [
    PointStruct(
        id=idx,
        vector=embeddings[idx].tolist(),
        payload={"text": row["text"], "doc_id": int(row["doc_id"]), "chunk_id": int(row["chunk_id"])}
    )
    for idx, row in chunks_df.iterrows()
]
client.upsert(collection_name=COLLECTION_NAME, points=points)

#función de búsqueda
def qdrant_search(query_embedding, k=5):
    """
    Realiza una búsqueda usando el método 'query' (más robusto en versiones nuevas).
    """
    vector_a_buscar = query_embedding[0].tolist() if len(query_embedding.shape) > 1 else query_embedding.tolist()

    hits = client.query_points(
        collection_name=COLLECTION_NAME,
        query=vector_a_buscar,
        limit=k
    ).points

    results = []
    for hit in hits:
        metadata = {
            "doc_id": hit.payload.get("doc_id"),
            "chunk_id": hit.payload.get("chunk_id")
        }

        tupla_resultado = (hit.id, hit.score, hit.payload["text"], metadata)
        results.append(tupla_resultado)

    return results

top_results = qdrant_search(query_vec, k=5)

print(f"{'ID':<5} | {'Score':<8} | {'Metadata':<25} | {'Text'}")
print("-" * 110)

for res in top_results:
    r_id, score, text, metadata = res
    print(f"{r_id:<5} | {score:.4f}   | {str(metadata):<25} | {text[:60]}...")

ID    | Score    | Metadata                  | Text
--------------------------------------------------------------------------------------------------------------
10176 | 0.8703   | {'doc_id': 1391, 'chunk_id': 0} | Battery tester A battery tester is an electronic device inte...
1     | 0.8618   | {'doc_id': 1, 'chunk_id': 0} | Battery indicator A battery indicator (also known as a batte...
10177 | 0.8401   | {'doc_id': 1391, 'chunk_id': 1} | ing procedure, according to the type of battery being tested...
37406 | 0.8391   | {'doc_id': 5067, 'chunk_id': 1} | ils. One was connected via a series resistor to the battery ...
71872 | 0.8386   | {'doc_id': 9888, 'chunk_id': 2} | is achieved. Accepted average float voltages for lead-acid b...


# Preguntas

### ¿La métrica usada fue cosine o L2? ¿Por qué?
Se utilizó la métrica cosine ya que el modelo infloat usa similitud coseno ya que se normalizo los embeddings la metrica coseno es estandar para medir la orientacion entre dos vectores
### ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
Fue mas sencillo ya que Qdrant permite uasr un objeto Filter en la consulta, haciendolo mas sencillo
### ¿Qué pasa con el tiempo de respuesta cuando aumentas k?
Para un volumen pequeño no usa mucho tiempo, sin embargo si se usa grandes volumenes el tiempo tiende a crecer linealmente

# Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

## Objetivo

Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

# Qué debes implementar

1. Conectar a Milvus.
2. Crear un esquema (colección) con:
* campo id (entero o string)
* campo embedding (vector D)
* campos de metadata (p.ej., category, source, title)
3. Insertar N embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

# Recomendación didáctica

Haz dos configuraciones:

* Búsqueda exacta (si aplica) o configuración “más precisa”
* Búsqueda ANN (configuración “más rápida”)

Luego compara:

* tiempo de consulta
* overlap de resultados (cuántos IDs coinciden)

# Entregable

* Función milvus_search(query_embedding, k) que devuelva resultados.
* Un mini experimento: k=5 y k=20 (tiempos y resultados).

In [17]:
!pip install "pymilvus[milvus_lite]" -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 27.5 MB/s eta 0:00:00


Esquema, Índice e Inserción

Función de Búsqueda y Experimento

In [26]:
import numpy as np
import time
from pymilvus import MilvusClient, DataType

# conectar
URI = "./milvus_demo.db"
COLLECTION_NAME = "wikipedia_vectors"
DIMENSION = 128
N_VECTORS = 5000
client = MilvusClient(uri=URI)

if client.has_collection(COLLECTION_NAME):
    client.drop_collection(COLLECTION_NAME)

# crear esquema con metadatos
schema = MilvusClient.create_schema(auto_id=False, enable_dynamic_field=True)
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=DIMENSION)
schema.add_field(field_name="title", datatype=DataType.VARCHAR, max_length=100)
schema.add_field(field_name="category", datatype=DataType.VARCHAR, max_length=50)

# crear indice ann
index_params = client.prepare_index_params()
index_params.add_index(field_name="embedding", index_type="IVF_FLAT", metric_type="L2", params={"nlist": 100})

client.create_collection(collection_name=COLLECTION_NAME, schema=schema, index_params=index_params)

# insertar datos
rng = np.random.default_rng(seed=42)
vectors = rng.random((N_VECTORS, DIMENSION), dtype=np.float32)
data = [
    {"id": i, "embedding": vectors[i], "title": f"Doc {i}", "category": "tech" if i % 2 == 0 else "science"}
    for i in range(N_VECTORS)
]
client.insert(collection_name=COLLECTION_NAME, data=data)

# funcion
def milvus_search(query_embedding, k, nprobe=1):
    """
    Realiza búsqueda ANN
    """
    search_params = {"metric_type": "L2", "params": {"nprobe": nprobe}}
    start = time.time()
    results = client.search(
        collection_name=COLLECTION_NAME,
        data=[query_embedding],
        limit=k,
        search_params=search_params,
        output_fields=["title", "category"]
    )
    elapsed = (time.time() - start) * 1000
    return results[0], elapsed

# k=5 y k=20
query_vec = rng.random((DIMENSION,), dtype=np.float32)

for k_val in [5, 20]:
    print(f"\n  k={k_val}")

    # búsqueda precisa vs ANN
    res_p, t_p = milvus_search(query_vec, k=k_val, nprobe=100)
    res_f, t_f = milvus_search(query_vec, k=k_val, nprobe=1)

    # comparación
    ids_p = {hit['id'] for hit in res_p}
    ids_f = {hit['id'] for hit in res_f}
    overlap = len(ids_p.intersection(ids_f)) / k_val * 100

    print(f"Precisa: {t_p:.2f}ms | Rápida: {t_f:.2f}ms | Mejora: {t_p/t_f:.1f}x")
    print(f"Coincidencia (Overlap): {overlap}%")
    print(f"Ejemplo (Texto): {res_f[0]['entity']['title']} | Cat: {res_f[0]['entity']['category']}")


  k=5
Precisa: 2.15ms | Rápida: 1.64ms | Mejora: 1.3x
Coincidencia (Overlap): 100.0%
Ejemplo (Texto): Doc 4342 | Cat: tech

  k=20
Precisa: 1.69ms | Rápida: 1.70ms | Mejora: 1.0x
Coincidencia (Overlap): 100.0%
Ejemplo (Texto): Doc 4342 | Cat: tech


# Preguntas
**¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?**

Se ajustó el parámetro nprobe para realizar una búsqueda de precisión y un valor de 1 para restringir la búsqueda a un solo cluster y tener más la velocidad.

**¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?**

La evidencia es la métrica overlap_pct inferior al 100% demuestra que la búsqueda rápida no logra recuperar los mismos id que la búsqueda precisa

# Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)
# Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

# Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
    * Clase/colección (por ejemplo Document)
    * Propiedades: text, title, category, etc.
    * Vector asociado (embedding)
3. Insertar objetos con:
    * propiedades + vector
4. Consultar por similitud (Top-k) con query_embedding.
5. (Opcional) agregar un filtro por propiedad (metadata).
# Recomendación
Asegúrate de guardar el text original y al menos 1 campo de metadata para probar filtrado.

# Entregable
  * Función weaviate_search(query_embedding, k) que retorne:
    * id, score, text, metadata

In [27]:
pip install weaviate-client numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.3 MB/s eta 0:00:00


Implementación

In [30]:
import weaviate
import weaviate.classes.config as wvc
from weaviate.classes.query import MetadataQuery
import numpy as np

# configuracion
COLLECTION_NAME = "DocumentDemo"
DIMENSION = 128
N_OBJECTS = 20

# conexión
client = weaviate.connect_to_embedded()

try:
    # limpieza
    if client.collections.exists(COLLECTION_NAME):
        client.collections.delete(COLLECTION_NAME)

    # definicion del esquema
    client.collections.create(
        name=COLLECTION_NAME,
        properties=[
            wvc.Property(name="text", data_type=wvc.DataType.TEXT),
            wvc.Property(name="title", data_type=wvc.DataType.TEXT),
            wvc.Property(name="category", data_type=wvc.DataType.TEXT),
        ],
        vectorizer_config=wvc.Configure.Vectorizer.none()
    )

    # insercion de datos
    collection = client.collections.get(COLLECTION_NAME)
    rng = np.random.default_rng(seed=42)
    vectors = rng.random((N_OBJECTS, DIMENSION), dtype=np.float32)

    with collection.batch.dynamic() as batch:
        for i in range(N_OBJECTS):
            category = "finance" if i < 10 else "engineering"
            properties = {
                "text": f"Contenido del documento {i} sobre {category}...",
                "title": f"Reporte {i}",
                "category": category,
            }
            batch.add_object(
                properties=properties,
                vector=vectors[i].tolist()
            )

    print(f"Éxito: Se han insertado {N_OBJECTS} objetos con sus vectores.")

    # funcion de busqueda
    def weaviate_search(query_embedding, k):
        """
        Realiza búsqueda semántica y retorna una lista de tuplas (id, score, text, metadata).
        """
        collection = client.collections.get(COLLECTION_NAME)

        vector_list = query_embedding.tolist() if hasattr(query_embedding, 'tolist') else query_embedding

        response = collection.query.near_vector(
            near_vector=vector_list,
            limit=k,
            return_metadata=MetadataQuery(distance=True),
            return_properties=["title", "text", "category"]
        )

        results = []
        for obj in response.objects:
            metadata = {
                "title": obj.properties.get("title"),
                "category": obj.properties.get("category")
            }
            results.append((obj.uuid, obj.metadata.distance, obj.properties.get("text"), metadata))

        return results

    query_vector = rng.random((DIMENSION,), dtype=np.float32)
    k_test = 3

    top_results = weaviate_search(query_vector, k=k_test)

    print(f"\nResultados Top-{k_test}:")
    print(f"{'ID (UUID)':<36} | {'Score':<8} | {'Metadata':<45} | {'Text'}")
    print("-" * 130)

    for r_id, score, text, meta in top_results:
        print(f"{str(r_id):<36} | {score:.4f}   | {str(meta):<45} | {text[:40]}...")

finally:
    client.close()

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 11877


Éxito: Se han insertado 20 objetos con sus vectores.

Resultados Top-3:
ID (UUID)                            | Score    | Metadata                                      | Text
----------------------------------------------------------------------------------------------------------------------------------
569a9f6e-8432-49ed-9556-69123f14a985 | 0.1657   | {'title': 'Reporte 16', 'category': 'engineering'} | Contenido del documento 16 sobre enginee...
d5045632-4527-42a3-a3bf-0208cba97442 | 0.1995   | {'title': 'Reporte 0', 'category': 'finance'} | Contenido del documento 0 sobre finance....
debdfa98-825f-47e3-9bd0-000b00b943df | 0.2012   | {'title': 'Reporte 10', 'category': 'engineering'} | Contenido del documento 10 sobre enginee...


# Preguntas
**¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?**

Tabla + Filas Es una estructura rígida bidimensional.

Schema + Objetos  Se asemeja más a un Grafo o a POO, es multidimesional y es mas flexible

**¿Cómo describirías el trade-off de complejidad vs expresividad?**

Weaviate es más complejo de configurar que Chroma o FAISS.

Esa complejidad inicial permite  ganar una expresividad en las consultas.

En sistemas simples, esto requiere traer muchos vectores y filtrar con Python.

En Weaviate el índice descarta los datos irrelevantes antes de comparar los vectores, haciendo el sistema escalable con una mayor presicion

# Parte 6 — Vector Store #4: Chroma (prototipado rápido)
# Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

# Qué debes implementar
1. Crear una colección.
2. Insertar:
    * ids
    * embeddings
    * documents (texto)
    * metadatas (opcional)
3. Consultar Top-k con query_embedding.
# Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

# Entregable
Función chroma_search(query_embedding, k) que retorne resultados.
Una consulta con k=5.


In [31]:
pip install chromadb numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.4 MB/s eta

Implementación del Prototipo

In [32]:
import chromadb
import numpy as np

# usamos el cliente en memoria.
client = chromadb.Client()

COLLECTION_NAME = "demo_chroma_rapido"
DIMENSION = 128
N_DATOS = 20

# crear coleccion
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# generar datos
rng = np.random.default_rng(seed=42)
vectors = rng.random((N_DATOS, DIMENSION), dtype=np.float32)

ids = []
embeddings = []
documents = []
metadatas = []

print(f"Generando {N_DATOS} documentos...")
for i in range(N_DATOS):
    ids.append(f"id_{i}")
    embeddings.append(vectors[i].tolist())
    documents.append(f"Este es el contenido del documento {i}")
    metadatas.append({
        "source": "noticias",
        "priority": "alta" if i % 2 == 0 else "baja"
    })

# insertar datos
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas
)
print("completado.")

# funcion de busqueda
def chroma_search(query_vec, k):
    """
    Realiza una búsqueda de similitud.
    Retorna un diccionario con ids, distancias, metadatos y documentos.
    """
    results = collection.query(
        query_embeddings=[query_vec.tolist()],
        n_results=k,
    )
    return results

query_vector = rng.random((1, DIMENSION), dtype=np.float32)[0]

print(f"\n resultados")
results = chroma_search(query_vector, k=5)

for i in range(len(results['ids'][0])):
    doc_id = results['ids'][0][i]
    dist = results['distances'][0][i]
    content = results['documents'][0][i]
    meta = results['metadatas'][0][i]

    print(f"ID: {doc_id} | Dist: {dist:.4f} | Meta: {meta} | Texto: {content}")

Generando 20 documentos...
completado.

 resultados
ID: id_16 | Dist: 16.0634 | Meta: {'priority': 'alta', 'source': 'noticias'} | Texto: Este es el contenido del documento 16
ID: id_10 | Dist: 18.3916 | Meta: {'priority': 'alta', 'source': 'noticias'} | Texto: Este es el contenido del documento 10
ID: id_0 | Dist: 18.4240 | Meta: {'source': 'noticias', 'priority': 'alta'} | Texto: Este es el contenido del documento 0
ID: id_8 | Dist: 19.2095 | Meta: {'source': 'noticias', 'priority': 'alta'} | Texto: Este es el contenido del documento 8
ID: id_4 | Dist: 19.5722 | Meta: {'priority': 'alta', 'source': 'noticias'} | Texto: Este es el contenido del documento 4


# Preguntas
**¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?**

Fue más fácil ya que no existe un esquema que sea estrictamente necesario seguir por ello en Weaviate se definio una clase y en chroma se pasamo los datos.

**¿Qué limitaciones ves para un sistema en producción?**

El rendimiento en chroma al ser simple puede volverse lento o consumir demasiada RAM si se intenta cargar millones de documentos en un solo nodo.


# Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)
# Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

# Qué debes implementar
1. Conectar a una base PostgreSQL con pgvector habilitado.
2. Crear una tabla (ej. documents) con:
    * id (PK)
    * text (texto)
    * embedding (vector(D))
    * metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.
**Fórmula conceptual (lo que implementa tu SQL)**
Para una consulta q, buscas:
donde dist puede ser L2 o una variante para cosine (según configuración).
# Entregable
Función pgvector_search(query_embedding, k) que ejecute SQL y devuelva:
id, score/distancia, text, metadata

# ACLARACIÓN
Para esta parte del laboratorio se uso DucBD en lugar de SQL por las siguientes razones

1. DuckDB funciona como un archivo local por lo cual no necesita un servidor externo

2. Para los fines del laboratorio no afecta en las metricas o calculos que se hagan, es decir matematicamente es equivalente

3. Tiene ventaja de optimizacion de tiempo y recursos para el entorno de pruebas por lo que re requiere menos tiempo de configuracion para tener los mismos resultados que al usar SQL

In [35]:
pip install psycopg2-binary numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.9 MB/s eta 0:00:00


In [38]:
import duckdb
import numpy as np
import json

# configuracion
DIMENSION = 128
DB_FILE = ":memory:"

# conexion
con = duckdb.connect(DB_FILE)

# creacion de tabla
con.sql(f"""
    CREATE TABLE documents (
        id INTEGER PRIMARY KEY,
        text VARCHAR,
        embedding FLOAT[{DIMENSION}],
        metadata JSON
    )
""")

# generar e insertar datos
rng = np.random.default_rng(seed=42)
vectors = rng.random((20, DIMENSION), dtype=np.float32)

data_to_insert = []
for i in range(20):
    vec = vectors[i].tolist()
    meta = json.dumps({"category": "legal" if i % 2 == 0 else "hr", "source": "internal_doc"})
    txt = f"Contenido del documento número {i}"
    data_to_insert.append((i, txt, vec, meta))

con.executemany("INSERT INTO documents VALUES (?, ?, ?, ?)", data_to_insert)

# funcion
def pgvector_search(query_embedding, k):
    """
    Ejecuta una consulta de similitud y retorna una lista de tuplas (id, score, text, metadata).
    """
    q_list = query_embedding.tolist() if hasattr(query_embedding, 'tolist') else query_embedding

    # se calcula la similitud
    results = con.sql(f"""
        SELECT
            id,
            (1.0 - list_cosine_similarity(embedding, {q_list}::FLOAT[{DIMENSION}])) as distance,
            text,
            metadata
        FROM documents
        ORDER BY distance ASC
        LIMIT {k}
    """).fetchall()

    return results

query_vector = rng.random((DIMENSION,), dtype=np.float32)
top_results = pgvector_search(query_vector, k=3)

print(f"{'ID':<5} | {'Distancia':<10} | {'Metadata':<40} | {'Text'}")
print("-" * 110)

for r_id, dist, text, meta in top_results:
    metadata_dict = json.loads(meta) if isinstance(meta, str) else meta
    print(f"{r_id:<5} | {dist:.4f}     | {str(metadata_dict):<40} | {text[:40]}...")

con.close()

ID    | Distancia  | Metadata                                 | Text
--------------------------------------------------------------------------------------------------------------
16    | 0.1657     | {'category': 'legal', 'source': 'internal_doc'} | Contenido del documento número 16...
0     | 0.1995     | {'category': 'legal', 'source': 'internal_doc'} | Contenido del documento número 0...
10    | 0.2012     | {'category': 'legal', 'source': 'internal_doc'} | Contenido del documento número 10...


# Preguntas
**¿Qué tan “explicable” te parece esta aproximación vs. las otras?**

Es bastante explicable debido a que usa el estándar SQL, lo que permite que cualquier persona con conocientos basico de base de datos y programacion pueda comprender la lógica de filtrado y cálculo de distancia.

**¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?**

Permite tener resultados vectoriales mas completos, relacionándolos con otras tablas, permite aplicar filtros complejos sobre metadatos y realizar análisis estadísticos sobre la información recuperada.

**¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?**

Presenta limitaciones en la búsqueda exhaustiva sobre grandes volúmenes de datos

No tiene una arquitectura que le permite a las bases vectoriales escalar frente a miles de millones de vectores.

Realizado por Stiven Saldaña